In [1]:
import torch
import torch.nn.functional as f
from sentence_transformers import sentencetransformer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

/home/anantajit/data/Bluesky-Discourse-Forecasting/.venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Using cuda


In [2]:
# data loader - use only Jan 20th
import pandas as pd 
from os import getcwd

print(f"Working Directory: {getcwd()}")

df = pd.read_parquet("../data/2025-01-20.parquet")
print(df.head())

Working Directory: /data/home/anantajit/Bluesky-Discourse-Forecasting/notebooks
                                                text                 createdAt
0                           Happy Birthday Tom Baker  2025-01-20T08:21:29.582Z
1  Been playing Astrobot this week-end with my da...  2025-01-20T08:21:30.788Z
2                    #MLKDay\n#MartinLutherKingJrDay  2025-01-20T08:21:30.771Z
3  I’m locked and loaded, fully customized!! Happ...  2025-01-20T08:21:29.713Z
4                                        No need 🙃😂🥰  2025-01-20T08:21:30.733Z


In [3]:
model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)
model.to("cuda:1")
# In case you want to reduce the maximum length:
model.max_seq_length = 300



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.60it/s]


In [9]:
from tqdm import tqdm

BLOCK_SIZE = 10_000

for i in tqdm(range(0, len(df), BLOCK_SIZE), desc="Block count"):
    slice = df[:][i:i + BLOCK_SIZE]

    tweet_embeddings = model.encode(slice['text'].tolist(), device=DEVICE, max_length=300)
    slice['embeddings'] = tweet_embeddings.tolist()
    
    slice.to_parquet(f"../data/embeddings/{i//BLOCK_SIZE:03d}.parquet")


Block count:   0%|          | 0/351 [00:00<?, ?it/s]

Block count: 100%|██████████| 351/351 [8:08:49<00:00, 83.56s/it]  
